## TFX Pipelines

#### Generate Examples

In [ ]:
# Temporary commands to unzip the zip file in Google Cloud Storage
# ! gsutil -m cp gs://text-analysis-323506/train_data/train_val.zip ./
# ! unzip train_val.zip
# ! gunzip *.csv.gz
# ! gsutil -m mv *.csv gs://text-analysis-323506/train_data/

In [ ]:
! pip3 install tfx==1.4.0
! pip install pyparsing==2.4.2

__Restart kernel__

In [1]:
import time
import os
import pickle

import tfx

import absl
import os
import tempfile
import time

import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
import tensorflow as tf

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.layers import TextVectorization

from tfx.components.common_nodes.importer_node import ImporterNode
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import example_gen_pb2
from tensorflow_metadata.proto.v0 import schema_pb2, statistics_pb2, anomalies_pb2
from tfx.components import StatisticsGen, CsvExampleGen, SchemaGen, ExampleValidator, Transform

from tfx.components import Trainer
from tfx.components import Transform
from tfx.components import Tuner
from tfx.dsl.components.base import executor_spec
from tfx.components.trainer import executor as trainer_executor

from tfx.proto import infra_validator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2

In [2]:
tf.__version__

'2.6.2'

In [3]:
tfx.__version__

'1.4.0'

In [10]:
DATA_ROOT = 'gs://text-analysis-323506/covertype'
ARTIFACT_STORE = os.path.join(os.sep, 'home', 'jupyter', 'artifact-store')
SERVING_MODEL_DIR=os.path.join(os.sep, 'home', 'jupyter', 'serving_model')

In [11]:
PIPELINE_NAME = 'covertype-analysis'
PIPELINE_ROOT = os.path.join('/home/jupyter/', PIPELINE_NAME, time.strftime("%Y%m%d_%H%M%S"))
os.makedirs(PIPELINE_ROOT, exist_ok=True)

In [12]:
context = InteractiveContext(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=None)

### CSV Example Generator

In [13]:
output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[        
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=4),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
    ]))

In [14]:
example_gen = CsvExampleGen(
    input_base=DATA_ROOT,
    output_config=output_config)

In [15]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "/home/jupyter/covertype-analysis/20211204_102754/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:5511886,xor_checksum:1638613661,sum_checksum:1638613661"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [16]:
examples_uri = example_gen.outputs['examples'].get()[0].uri

In [17]:
tfrecord_filenames = [os.path.join(examples_uri, 'Split-train', name)
                      for name in os.listdir(os.path.join(examples_uri, 'Split-train'))]

In [18]:
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

2021-12-04 10:28:24.881727: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [19]:
for tfrecord in dataset.take(2):
    example = tf.train.Example()
    example.ParseFromString(tfrecord.numpy())
    for name, feature in example.features.feature.items():
        if feature.HasField('bytes_list'):
            value = feature.bytes_list.value
        if feature.HasField('float_list'):
            value = feature.float_list.value
        if feature.HasField('int64_list'):
            value = feature.int64_list.value
        print('{}: {}'.format(name, value))
    print('******')

Vertical_Distance_To_Hydrology: [101]
Slope: [9]
Wilderness_Area: [b'Commanche']
Horizontal_Distance_To_Hydrology: [648]
Horizontal_Distance_To_Fire_Points: [1871]
Horizontal_Distance_To_Roadways: [757]
Elevation: [3142]
Hillshade_9am: [223]
Aspect: [183]
Hillshade_3pm: [157]
Cover_Type: [1]
Soil_Type: [b'C7757']
Hillshade_Noon: [247]
******
Hillshade_3pm: [105]
Hillshade_9am: [245]
Aspect: [124]
Slope: [16]
Vertical_Distance_To_Hydrology: [9]
Soil_Type: [b'C2704']
Wilderness_Area: [b'Cache']
Hillshade_Noon: [227]
Elevation: [1967]
Horizontal_Distance_To_Fire_Points: [451]
Cover_Type: [2]
Horizontal_Distance_To_Hydrology: [60]
Horizontal_Distance_To_Roadways: [124]
******


2021-12-04 10:28:24.986417: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


#### Train and eval datasets have been created properly !

### Statistics Generator

In [20]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

In [21]:
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 16
        uri: "/home/jupyter/covertype-analysis/20211204_102754/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [22]:
context.show(statistics_gen.outputs['statistics'])

### Schema Generator

In [23]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)

In [24]:
context.run(schema_gen)

I1204 10:28:31.451728 28156 rdbms_metadata_access_object.cc:686] No property is defined for the Type


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "/home/jupyter/covertype-analysis/20211204_102754/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [25]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,single,-
'Cover_Type',INT,required,single,-
'Elevation',INT,required,single,-
'Hillshade_3pm',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-
'Horizontal_Distance_To_Hydrology',INT,required,single,-
'Horizontal_Distance_To_Roadways',INT,required,single,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


In [26]:
schema_proto_path = '{}/{}'.format(schema_gen.outputs['schema'].get()[0].uri, 'schema.pbtxt')
schema = tfdv.load_schema_text(schema_proto_path)

In [27]:
tfdv.set_domain(schema, 'Cover_Type', schema_pb2.IntDomain(name='Cover_Type', min=0, max=6, is_categorical=True))
tfdv.set_domain(schema, 'Slope',  schema_pb2.IntDomain(name='Slope', min=0, max=90))

tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,single,-
'Cover_Type',INT,required,single,min: 0; max: 6
'Elevation',INT,required,single,-
'Hillshade_3pm',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-
'Horizontal_Distance_To_Hydrology',INT,required,single,-
'Horizontal_Distance_To_Roadways',INT,required,single,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


In [28]:
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,single,-
'Cover_Type',INT,required,single,min: 0; max: 6
'Elevation',INT,required,single,-
'Hillshade_3pm',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-
'Horizontal_Distance_To_Hydrology',INT,required,single,-
'Horizontal_Distance_To_Roadways',INT,required,single,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


#### Write schema to new file

In [ ]:
schema_dir = os.path.join(ARTIFACT_STORE, 'schema')
tf.io.gfile.makedirs(schema_dir)
schema_file = os.path.join(schema_dir, 'schema.pbtxt')

tfdv.write_schema_text(schema, schema_file)

!cat {schema_file}

### Schema Importer

In [30]:
schema_importer = tfx.dsl.components.common.importer.Importer(
      source_uri=schema_dir,
      artifact_type=tfx.types.standard_artifacts.Schema).with_id(
          'schema_importer') 

In [ ]:
context.run(schema_importer)

In [32]:
context.show(schema_importer.outputs['result'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,single,-
'Cover_Type',INT,required,single,min: 0; max: 6
'Elevation',INT,required,single,-
'Hillshade_3pm',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-
'Horizontal_Distance_To_Hydrology',INT,required,single,-
'Horizontal_Distance_To_Roadways',INT,required,single,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


### ExampleValidator

In [33]:
example_validator = ExampleValidator(    
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_importer.outputs['result']).with_id(
          'example_validator') 

In [ ]:
context.run(example_validator)

In [ ]:
train_uri = example_validator.outputs['anomalies'].get()[0].uri
train_anomalies_filename = os.path.join(train_uri, "Split-train/SchemaDiff.pb")
!cat $train_anomalies_filename

In [36]:
context.show(example_validator.outputs['anomalies'])

### Transform

In [37]:
TRANSFORM_MODULE = 'preprocessing.py'

In [33]:
data = pd.read_csv(data_path)

#### Create a tokenizer
This should be moved to a custom component. this new component will be responsible for creating a tokenizer and also converting string input to an integer format

In [34]:
tokenizer = Tokenizer(num_words=TOP_K)

In [35]:
tokenizer.fit_on_texts(data['input'])

In [46]:
MAX_SEQUENCE_LENGTH = 250
VOCAB_SIZE = 20000

int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

In [47]:
int_vectorize_layer.adapt(list(data['input']))

In [ ]:
with open('./vectorizer_layer.pickle', 'wb') as handle:
    pickle.dump(int_vectorize_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [49]:
! gsutil -m cp vectorizer_layer.pickle gs://text-analysis-323506/

Copying file://vectorizer_layer.pickle [Content-Type=application/octet-stream]...
- [1/1 files][ 33.9 MiB/ 33.9 MiB] 100% Done                                    
Operation completed over 1 objects/33.9 MiB.                                     


In [68]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_importer.outputs['result'],
    module_file=TRANSFORM_MODULE,
    custom_config={'tokenizer_path': 'gs://text-analysis-323506/vectorizer_layer.pickle'})

In [71]:
context.run(transform)

Processing /home/jupyter/sentiment-analysis/20211202_175236/_wheels/tfx_user_code_Transform-0.0+73f68bd3f6adf04d6431dc80024bbdad2ef65dccb3ada9fc7d62423eebc1d068-py3-none-any.whl
Processing /home/jupyter/sentiment-analysis/20211202_175236/_wheels/tfx_user_code_Transform-0.0+73f68bd3f6adf04d6431dc80024bbdad2ef65dccb3ada9fc7d62423eebc1d068-py3-none-any.whl
Processing /home/jupyter/sentiment-analysis/20211202_175236/_wheels/tfx_user_code_Transform-0.0+73f68bd3f6adf04d6431dc80024bbdad2ef65dccb3ada9fc7d62423eebc1d068-py3-none-any.whl


Copying gs://text-analysis-323506/vectorizer_layer.pickle...
- [1/1 files][ 33.9 MiB/ 33.9 MiB] 100% Done                                    
Operation completed over 1 objects/33.9 MiB.                                     


TypeError: Failed to convert object of type <class 'tensorflow.python.framework.sparse_tensor.SparseTensor'> to Tensor. Contents: SparseTensor(indices=Tensor("inputs_copy:0", shape=(None, 2), dtype=int64), values=Tensor("inputs_1_copy:0", shape=(None,), dtype=string), dense_shape=Tensor("inputs_2_copy:0", shape=(2,), dtype=int64)). Consider casting elements to a supported type.

In [38]:
transform.outputs['transformed_examples'].get()[0].uri

'/home/jupyter/sentiment-analysis/20211201_173227/Transform/transformed_examples/6'

In [39]:
os.listdir(transform.outputs['transformed_examples'].get()[0].uri)

['Split-train', 'Split-eval']

In [40]:
transform_uri = transform.outputs['transformed_examples'].get()[0].uri
tfrecord_filenames = [os.path.join(transform_uri,  'Split-train', name)
                      for name in os.listdir(os.path.join(transform_uri, 'Split-train'))]

In [41]:
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
for tfrecord in dataset.take(2):
    example = tf.train.Example()
    example.ParseFromString(tfrecord.numpy())
    for name, feature in example.features.feature.items():
        if feature.HasField('bytes_list'):
            value = feature.bytes_list.value
        if feature.HasField('float_list'):
            value = feature.float_list.value
        if feature.HasField('int64_list'):
            value = feature.int64_list.value
    print('{}: {}'.format(name, value))
    print('******')

labels_xf: [1]
******
input_xf: [b'great book liked book sarah plain tall authors name patricia maclachen characters names caleb sarah anna papa though book interestingit first started caleb siting fire asking questions mama singing songs anna explaining born mama died sarah answered papas letter came live sarah taught caleb swim anybody gets book really enjoy']
******


### Trainer

In [45]:
TRAINER_MODULE_FILE = 'model.py'

In [46]:
trainer = Trainer(
    custom_executor_spec=executor_spec.ExecutorClassSpec(trainer_executor.GenericExecutor),
    module_file=TRAINER_MODULE_FILE,
    transformed_examples=transform.outputs['transformed_examples'],
    schema=schema_importer.outputs['result'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=5000),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=1000))

In [ ]:
context.run(trainer)